<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/river_width.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

In [1]:
import ee
import geemap
from geemap.algorithms import river

In [2]:
geemap.ee_initialize()

In [3]:
img_id = 'LC08_L1TP_022034_20130422_20170310_01_T1'

In [4]:
river.rwc(img_id)

LC08_L1TP_022034_20130422_20170310_01_T1 will be exported to  as shp file


In [6]:
in_csv = 'https://raw.githubusercontent.com/giswqs/geemap/master/examples/data/rwc_batch_input.csv'

In [7]:
river.rwc_batch(in_csv)

LT05_L1TP_017036_20060502_20160910_01_T1 will be exported to  as shp file
LT05_L1TP_017036_20060502_20160910_01_T1 will be exported to  as shp file
LT05_L1TP_017036_20061212_20160908_01_T1 will be exported to  as shp file
